# overlay segmentation mask on microscopy image

needs:
* grayscale microscopy or lensfree microscopy iamge
* and a foreground mask

In [ ]:
import cv2
import numpy as np

import colormaps as cmaps

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def colorMapIndex(clustersize) :
    index = 0
    if(clustersize > cmin + 1.0) :
        index = min(int(255* np.log(clustersize-cmin)/lncmax), 255)
    return(index)


def compOverlayImage(image,mask,transparancylevel):
    im2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    foreGroundContourList = []
    backGroundContourList = []

    i = 0
    for cnt in contours:
        if(hierarchy[0][i][3] == -1) :  # i.e., if it is a foreground object
            area = cv2.contourArea(cnt)
            holeArea = 0
            holeIndex = hierarchy[0][i][2]
            while holeIndex != -1 :
                holeArea += cv2.contourArea(contours[holeIndex])
                holeIndex = hierarchy[0][holeIndex][0]
            area -= holeArea
            foreGroundContourList += [(cnt, area)]
        else :
            area = cv2.contourArea(cnt)
            backGroundContourList += [(cnt, area)]
        i += 1



    fgSorted = sorted(foreGroundContourList, key=lambda x: x[1])
    bgSorted = sorted(backGroundContourList, key=lambda x: x[1])

    lncmax = np.log(0.15*1944.0*2592.0)   # Clustersize corresponding to the maximum of 
                                    # the color scale (=10% of the image area)
    cmin = 2000.0                   # Clustersize corresponding to the minimum of 
                                    # the color scale

    palette = cmaps._plasma_data
    palette = palette[::-1]
    colorMask = np.zeros(image.shape, np.uint8)
    for entry in reversed(fgSorted) :
        fgContour = np.array(entry[0]).reshape(-1,1,2).astype(np.int32)
        index = 255 - colorMapIndex(entry[1])
        R = int(255*palette[index][0])
        G = int(255*palette[index][1])
        B = int(255*palette[index][2])
        cv2.drawContours(colorMask, [fgContour], 0, (B,G,R), cv2.FILLED)
        fgArea = cv2.contourArea(fgContour)
        for entry2 in reversed(bgSorted) :
            bgContour = np.array(entry2[0]).reshape(-1,1,2).astype(np.int32)
            bgArea = cv2.contourArea(bgContour)
            if(bgArea < fgArea) :
                cv2.drawContours(colorMask, [bgContour], 0, (0,0,0), cv2.FILLED)
    output = cv2.addWeighted(colorMask, transparancylevel, image, 1.0-transparancylevel, 0, None)
    return output 

In [ ]:
transparancylevel = 0.5
image = cv2.cvtColor(raw,cv2.COLOR_GRAY2RGB)

output = compOverlayImage(image,img_mask,transparancylevel)

plt.figure(figsize=(10,10))
plt.imshow(output)
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.title('overlay Image')
plt.show()